# Tutorial of NvDocker interfaces

In [10]:
import socket
from dsd.sys.docker import NvDocker

try:
    socket.gethostbyname('dockerhost')
    base_url = 'http://dockerhost:3476'
except socket.error:
    base_url = 'http://localhost:3476'
    
nvd = NvDocker(base_url=base_url)

# Gpu global informations

In [11]:
globalInfo = nvd.gpuGlobalInfo()

print 'Driver Version:', globalInfo['DriverVersion']
print 'CUDA Version:', globalInfo['CudaVersion']

Driver Version: 352.39
CUDA Version: 7.5


# Gpu status

In [12]:
gpuInfo = nvd.gpuInfo()

for gpu in gpuInfo:
    print '#' * 20
    print 'Model:', gpu['Model']
    print 'Path:', gpu['Path']
    print 'Memory:', gpu['MemoryUsed'], '/', gpu['MemoryTotal'], 'MB'
    
    print 'Utilization GPU:', gpu['UtilizationGpu']
    print 'Utilization Memory:', gpu['UtilizationMemory']
    print 'Temperature:', gpu['Temperature']
    
    for p in gpu['Processes']:
        print ' '*8, 'PID:', p['PID'], 'Name:', p['Name'], 'MemoryUsed:', p['MemoryUsed'], 'MB'

####################
Model: Tesla K40m
Path: /dev/nvidia0
Memory: 1446 / 11519 MB
Utilization GPU: 69
Utilization Memory: 34
Temperature: 47
         PID: 20401 Name: /usr/bin/python MemoryUsed: 73 MB
         PID: 30093 Name: python MemoryUsed: 1347 MB
####################
Model: Tesla K40m
Path: /dev/nvidia1
Memory: 162 / 11519 MB
Utilization GPU: 0
Utilization Memory: 0
Temperature: 37
         PID: 30093 Name: python MemoryUsed: 137 MB
